# Model Exploration
In this file, I will be investigating the PAWS model for Stackelberg Security Games and Wildlife Conservation: https://projects.iq.harvard.edu/files/teamcore/files/project_paws_publication_01_teamcore_yang_et_al_2014.pdf. In this paper, the authors design a stackelberg game where the two agents are the rangers and the poachers. They play a turn based game where the rangers first setup a patrol strategy. The poachers can then observe this strategy and come up with a poaching strategy in return. The patrol/poaching areas are discritized by splitting the map into a grid, and a strategy is defined by assigning the limited number of rangers to a cell of the grid. From here, utility is assigned to the different outcomes of people present combined with if there are endangered animals present, with positive utility given to catching and stopping poachers and negative utility to poachers without a ranger. Using this score of the board, the rangers then update their strategy to reflect the observed poacher behavior. This cycle repeats with the goal of rangers encountering more poachers.

For our approach, we want to use this same model with the agents being the trappers and the invasive species we are looking to capture. To make this modification, we first need to change the utility structure such that we reward traps capturing the invasive species and disincentivize traps being placed somewhere where they dont catch an invasive animal. To do this, we first need to understand how the original paws implementation works.

## PAWS Structural Outline and the modifications we will need to make

From my 91r work junior fall, I have put together a rough outline of the PAWS codebase as it works as a flask app in python. To run the model, researchers do the following things:

1. **Setup the Flask App and log**: Sets up the overhead and logging needed to track the execution of the flask app containing this software. This portion also establishes connection to an Azure server containing the data needed to operate the program. This portion, relevant for deployment and scaling, will not be relevant to our implementation.
2. **Data Input and Breakdown**: In the PAWS design, researchers take data keys from a JSON file that then allow them to access the data of interest from the cloud. This data comes in 3 parts:
    - shapes, or the shapefiles representing the patrolled park (i.e. park boundaries or locations of different features of the landscape like rivers and roads)
    - rasters, or the raster files providing supplemental information about a patrolled park (i.e. elevation of different areas in a park, land cover)
    - patrol_observations, or a csv containing information about the different patrols used for prediction.
    
    I have a data dictionary for both the JSON file and patrol_observations file to understand what data goes where. Generally though, for our implementation we will be cutting the JSON mapping to the cloud and grabbing the right datasets and instead creating a function to setup the data we need locally.

3. **Data Validation and Error Handling**: this section allows us to verify that necessary features of a given piece of data holds before attempting to process the data for prediction. Problems are broken down into Warnings, Errors, and FatalErrors that will log and kill the program if fatal. Understanding and updating these checks based on the modified reward and data will be important as we test and develop our modifications.
4. **Preprocess**: This does all of the data pre-processing, where we grab the boundary, clean the data, get shapefiles, distances, and rasters, calculate effort, and compute illegal activity. Some data verification does happen in this file, so checking if this data verification can be refactored into the data validation section could be useful. For our implementation, I think the preprocessing will look quite similar with the exception of how to process the reward-defining activity.
5. **Consolidate**: This consolidates all of the data pieces into a single dataframe for the model to use. The PAWS implementaton of consolidate contains some repeated processes that already exist in preprocess, so for our implementation it will be nice to remove those repeat processes. Besides this, consolidation should look extremely similar for our implementation. Additionally, it may be worth working with everything in a more final format from the beginning to avoid unnecessary overhead, but this would be future work after an initial implementation.
6. **Predict**: Here, taking this single dataframe, we ultimately predict the locations for our next round of patrols. For our implementation, the prediction should be identical, so long as the modified reward is captured within the data. If not, then the actual prediction schematic may need to update its scoring to reflect rewarding the successful traps. 

## Invasive Species Modifications

Working on this, I want to start with the initial framework of the prediction to better understand what it uses and therefore what it needs for modifications. 

In [13]:
## imports ##

In [14]:
## log handling ##

class Log:
    def __init__(self, filename="../Logs/log.txt"):
        self.filename = filename
        # Clear the file's contents if it exists
        with open(self.filename, "w") as file:
            pass

    def write_line(self, line):
        with open(self.filename, "a") as file:
            file.write(line + "\n")

In [15]:
## data input and breakdown ##

def traps_get_data(log):
    pass

In [16]:
## data validation and error handling ##

def traps_validate_data(log):
    pass

In [17]:
## preprocess ##

def traps_preprocess(log):
    pass

In [18]:
## consolidate ##

def traps_consolidate(log):
    pass

In [28]:
## predict ##
# imports
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from os import getenv, path
from functools import partial
from multiprocessing import get_context
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import LinearSVC, SVC
from gpc import GaussianProcessClassifier

import copy

# global vars
MULTIPROCESSING_POOL_SIZE_ENV_VAR='MULTIPROCESSING_POOL_SIZE'
POSITIVE_LABEL = 1      # how a positive label is encoded in the data
RANDOM_SEED = None        # could be None
N_JOBS = 1 # -1 to use max
# parameters for bagging classifier
NUM_ESTIMATORS = 32 #32 #50
MAX_SAMPLES = 0.8
MAX_FEATURES = .5
# verbose output if == 1
VERBOSE = 0

TRAIN_SECTION = 1


# define model class to train and predict from
class SSGModel:
    # setup params for the class
    def __init__(self, classifier_model, classifier_count, pred_start_timestep, log):
        self.classifier_model = classifier_model
        self.classifier_count = classifier_count
        self.pred_start_timestep = pred_start_timestep
        self.log = log
        self.trap_thresholds = None
        self.classifiers = None
        self.weights = None         # weights for classifiers


    # training classifiers based on the environment
    def train_single_classifier(self, patrol_threshold, use_balanced, train_x, train_y, train_effort):
        # reduce to size desired
        train_y = np.squeeze(train_y)
        
        # get index of where to cut off points to analyze where either of these expressions holds (positive label or effort is beyond/at threshold)
        idx = np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL))[0]

        # if idx has no points for a given training threshold nothing can happen
        if idx.size == 0:
            print('no training points found for threshold = {}'.format(patrol_threshold))
            return None

        # filter data for given idx 
        train_x_filter = train_x[idx, :]
        train_y_filter = train_y[idx]

        print('filtered data: {}. num positive labels {}. threshold {}'.format(train_x_filter.shape, np.sum(train_y_filter), patrol_threshold))

        # filtered labels have no positive hits, threshold not useful to train on.
        if np.sum(train_y_filter) == 0:
            print('no positive labels in this subset of the training data. skipping threshold {}'.format(patrol_threshold))
            return None

        # fit training data, get classifier and fit function according to method using get_classifier fn
        # TODO Fill in to pick classifier by method
        # classifier = get_classifier(use_balanced, method)
        
        if self.method == 'random_forest':
            classifier = RandomForestClassifier(n_estimators=NUM_ESTIMATORS,
                    criterion='gini', max_depth=None, min_samples_split=2,
                    min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                    max_features=MAX_FEATURES, max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    bootstrap=True, oob_score=False, n_jobs=N_JOBS,
                    random_state=RANDOM_SEED, verbose=VERBOSE,
                    warm_start=False, class_weight=None)

        else:
            if self.method == 'gaussian_processes':
                kernel = 1.0 * RBF(length_scale=1.0)
                base_estimator = GaussianProcessClassifier(kernel=kernel, random_state=RANDOM_SEED, warm_start=True, max_iter_predict=100, n_jobs=-1)
            elif self.method == 'svm':
                base_estimator = SVC(gamma='auto', random_state=RANDOM_SEED)
            elif self.method == 'linear-svc':
                base_estimator = LinearSVC(max_iter=5000, random_state=RANDOM_SEED)
            elif self.method == 'decision_tree':
                base_estimator = tree.DecisionTreeClassifier(random_state=RANDOM_SEED)
            else:
                raise Exception('method \'{}\' not recognized'.format(self.method))

            if self.method == 'gaussian_processes':
                classifier = base_estimator
            
            elif use_balanced:
                # balanced bagging classifier used for datasets with strong label imbalance
                classifier = BalancedBaggingClassifier(base_estimator=base_estimator,
                    n_estimators=NUM_ESTIMATORS, max_samples=MAX_SAMPLES,
                    max_features=MAX_FEATURES,
                    bootstrap=True, bootstrap_features=False,
                    oob_score=False, warm_start=False,
                    sampling_strategy='majority', #sampling_strategy=0.8,
                    replacement=True, n_jobs=N_JOBS,
                    random_state=RANDOM_SEED, verbose=VERBOSE)
            else:
                # non-balanced bagging classifier used for other datasets
                classifier = BaggingClassifier(base_estimator=base_estimator,
                        n_estimators=NUM_ESTIMATORS, max_samples=MAX_SAMPLES,
                        max_features=MAX_FEATURES,
                        bootstrap=True, bootstrap_features=False,
                        oob_score=False, warm_start=False, n_jobs=N_JOBS,
                        random_state=RANDOM_SEED, verbose=VERBOSE)
        
        classifier.fit(train_x_filter, train_y_filter)

        # return fitted classifier
        return classifier
            
        
    # define full training
    def train(self, predict_section, predict_section_test, features_raw, features, feature_names, labels, trap_effort, section_col, input_static_feats, test_temp=None, test_precip=None,use_balanced=False):
        
        predict_timestep = self.pred_start_timestep

        # split data for training and predicting: use all data before specified (predict_year, predict_section)
        train_idx = np.where(np.logical_or(features_raw['timestep'] < predict_timestep,
            np.logical_and(features_raw['timestep'] == predict_timestep, features_raw['section'] < predict_section_test)))[0]

        train_x = features[train_idx, :]
        train_y = labels[train_idx]
        train_trap_effort = trap_effort[train_idx]

        if predict_section == 0:
            prev_timestep = predict_timestep - 1
            num_section = np.max(section_col)
            prev_section = num_section
        else:
            prev_timestep = predict_timestep
            prev_section = predict_section - 1

        self.log.write_line("training/predicting data split complete. Test at timestep {} in section {}".format( predict_timestep, predict_section))

        # set up data arrays #
        # get past trapping effort for the test section
        prev_section_idx = np.where(np.logical_and(features_raw['timestep'] == prev_timestep, features_raw['section'] == prev_section))
        past_trap_effort = trap_effort[prev_section_idx]

        prev_section_spatial_id = features_raw['spatial_id'].values[prev_section_idx]
        trap_effort_df = pd.DataFrame({'spatial_id': prev_section_spatial_id,
                                            'past_trap_effort': past_trap_effort})

        # create features array and add in past_patrol_effort
        # TODO uses spatial_id to join, will dataframe we use still have this?
        predict_x_df = input_static_feats.join(trap_effort_df.set_index('spatial_id'), on='spatial_id', how='left')
        predict_x_df['past_patrol_effort'].fillna(0, inplace=True)

        # add climate info
        # TODO determine if climate info will still be used
        if test_temp is not None and test_precip is not None:
            predict_x_df['temp']   = test_temp * np.ones(input_static_feats.shape[0])
            predict_x_df['precip'] = test_precip * np.ones(input_static_feats.shape[0])

        # arrange columns to match training data
        store_columns = predict_x_df[['spatial_id', 'x', 'y']]
        predict_x_df.drop(columns=['spatial_id', 'x', 'y'], inplace=True)
        predict_x_df = predict_x_df[feature_names]
        predict_x = predict_x_df.values
        
        # normalize data
        scaler = StandardScaler()
        # fit only on training data
        scaler.fit(train_x)
        # apply normalization to training and test data
        train_x = scaler.transform(train_x)
        predict_x = scaler.transform(predict_x)

        # train classifiers #
        self.log.write_line("training classifiers on {} points...".format(train_x.shape)) # iware
        train_start_time = time.time()
        self.trap_thresholds = self.get_patrol_thresholds(train_trap_effort)
        vote_power = np.identity(self.num_classifiers)                           #get_vote_matrix: identity matrix
        vote_qual = np.ones((self.num_classifiers, self.num_classifiers))
        # create combined vote matrix
        self.weights = np.multiply(vote_power, vote_qual)
        # normalize column-wise
        self.weights =  self.weights /  self.weights.sum(1)[:,None]
        self.log.write_line("finished calculating weights.")
        #train classifiers
        # self.classifiers = self.train_classifiers(self.patrol_thresholds, train_x, train_y, train_trap_effort, use_balanced)
        self.log.write_line("begin setup multi-classifier training.")
        self.classifiers = []
        pool_size = getenv(MULTIPROCESSING_POOL_SIZE_ENV_VAR, None)
        self.log.write_line('Training ' + str(len(self.trap_thresholds)) + ' classifiers with a pool size of ' + str(pool_size))
        with get_context("spawn").Pool(processes=int(pool_size)) as pool:
            # time process
            start_time = time.time()
            # get classifiers by training each single classifier
            self.classifiers = pool.map(partial(self.train_single_classifier, use_balanced, train_x=train_x, train_y=train_y, train_effort=train_trap_effort), self.trap_thresholds)
            self.log.write_line("done training all classifiers.")
        # keep individual classifier training abstracted to support different models
        total_train_time = time.time() - train_start_time
        self.log.write_line("done training classifiers. total train time {:.3f}".format(total_train_time))
        
        return train_x, predict_x, store_columns

    
    #define predicting
    def predict(self, predict_section, train_x, predict_x, store_columns):
        # def make_predictions(self, predict_section, train_x, predict_x, store_columns):
        # ----------------------------------------------
        # run classifiers to get set of predictions
        # ----------------------------------------------
        # intiialize array to store predictions from each classifier
        predict_year = self.pred_start_timestep
        self.log.write_line('making predictions on year {} section {}... {} points'.format(predict_year, predict_section, predict_x.shape))
        final_predictions = np.zeros((predict_x.shape[0], self.num_classifiers))
        if self.method == 'gaussian_processes' or self.method == 'random_forest':
            final_variances = np.zeros((predict_x.shape[0], self.num_classifiers))


        # make predictions with each classifier
        for i in range(self.num_classifiers):

            self.log.write_line('iteration ' + str(i))

            # this classifier had no training points, so we skip it
            if self.classifiers[i] is None:
                final_predictions[:, i] = np.zeros((final_predictions.shape[0]))
                continue

            if self.method == 'gaussian_processes' or self.method == 'random_forest':
                if self.method == 'random_forest':
                    curr_predictions, curr_variances = self.classifiers[i].predict_proba(predict_x, return_var=True, train_x=train_x)
                else:
                    curr_predictions = self.classifiers[i].predict_proba(predict_x)
                    curr_variances = self.classifiers[i].predict_var(predict_x)

                print('variance min {} max {}'.format(np.min(curr_variances), np.max(curr_variances)))

                final_variances[:, i] = curr_variances
            else:
                curr_predictions = self.classifiers[i].predict_proba(predict_x)

            # curr_predictions has two columns, first column for probability of 0, second col for prob of 1
            # if on classifier iter 8 and 9, number of filtered datapoints = number of positive illegal activity
            # this is because for higher patrol thresholds, there may not be any datapoints from train_x that have greater patrol effort
            # so for the np.where constructing filtered data, the selected data is just the illegal data,
            # so we can predict 100% confidence those have illegal instances
            # so for iter 8 and 9 it's just 1 column because the prob is all 1 so it'll be e.g. (97,1) rather than (97,2)

            # for (n,1) case, where all n probabilities are 1
            if np.size(curr_predictions) == curr_predictions.shape[0]:
                curr_predictions = np.ravel(curr_predictions)
            # for (n,2) case, first column for probability of 0, second col for prob of 1
            else:
                print('in')
                curr_predictions = curr_predictions[:, 1]   # probability of positive label

            final_predictions[:, i] = curr_predictions


        self.log.write_line("finished making predictions. Now saving to csv...")

        # save out predictions to CSV
        predictions_df = pd.DataFrame(data=final_predictions, columns=['threshold={}'.format(thresh) for thresh in self.patrol_thresholds])
        predictions_df = pd.concat([store_columns, predictions_df], axis=1)

        # save out variances to CSV
        variances_df = None
        if self.method == 'gaussian_processes' or self.method == 'random_forest':
            self.log.write_line("Now saving variances csv...")
            variances_df = pd.DataFrame(data=final_variances, columns=['threshold={}'.format(thresh) for thresh in self.patrol_thresholds])
            variances_df = pd.concat([store_columns, variances_df], axis=1)
        self.log.write_line("returning predictions.")
        return predictions_df, variances_df


# TODO: requires data input as 2 separate csvs:
    # one of the features for the given region of predictive interest 
    # one of the labels for the given region of predictive interest
# inputs to this function:
    # log: instance of the log class. Allows for logging information about a run to a specified text file
    # features_file_path: relative path from this script to the features csv file
    # labels_file_path: relative path from this script to the labels csv file
    # static_features_path: relative path from this script to the static_features csv file
    # classifier_model: name of the type of classifier I want to use
    # pred_start_timestep: int value of timestep in input data to split training and predicting data
    # write_dir: directory to write predictions to
def traps_predict(log, features_file_path, labels_file_path, static_features_path, classifier_model, pred_start_timestep, write_dir):
    # find number of sections (here split by month) --> TODO may get rid of
    # section_count = int(12 / temporal_training_resolution_month_count)
    
    # gather the data into its final format as a pd dataframe:
    # get post-processed features csv (raw features)
    features_raw = pd.read_csv(features_file_path)
    # get post-processed labels (raw labels)
    labels_raw = pd.read_csv(labels_file_path)
    # get static features
    geo_features_csv = pd.read_csv(static_features_path)
    log.write_line("final data read in successfully.")

    # generate the following:
        # patrol_effort: features_raw['current_patrol_effort'].values, direct value from raw features csv
        # section_col: features_raw['section'].values, direct value from raw features csv
        # features_raw: overwrite pulled raw features csv that cuts the current patrol effort and the first column (TODO what is that first column, perhaps empty overhead?)
        # features: copy of overwritten features_raw that removes global_id, year, section, spatial_id, x, y and only contains the values of the remaining content
        # feature_names: object that contains the names of all the remaining values captured in features
        # labels: modified original labels that removes the first column, global_id, year, section, spatial_id, x, y, and only contains the values of the remaining content
    # TODO mod to be trapping effort
    trap_effort = features_raw['current_trapping_effort'].values
    section_col   = features_raw['section'].values
    # year_col      = features_raw['timestep'].values
    # drop first column and the current_patrol_effort after grabbing the values and the corresponding first col label
    features_raw.drop(columns=features_raw.columns[0], inplace=True)
    features_raw.drop(columns='current_trapping_effort', inplace=True)
    labels_raw.drop(columns=labels_raw.columns[0], inplace=True)
    # get rid of following column labels 
    labels_raw.drop(columns='global_id', inplace=True)
    labels_raw.drop(columns='timestep', inplace=True)
    labels_raw.drop(columns='section', inplace=True)
    labels_raw.drop(columns='spatial_id', inplace=True)
    labels_raw.drop(columns='x', inplace=True)
    labels_raw.drop(columns='y', inplace=True)
    # make a copy of the features that removes the removed labels from ebfore
    features = copy.copy(features_raw)
    features.drop(columns='global_id', inplace=True)
    features.drop(columns='timestep', inplace=True)
    features.drop(columns='section', inplace=True)
    features.drop(columns='spatial_id', inplace=True)
    features.drop(columns='x', inplace=True)
    features.drop(columns='y', inplace=True)
    # grab the values from both edited objects 
    features = features.values
    labels   = labels_raw.values
    # convert column labels into a list to grab the feature names and remove the irrelevant feature names, grab label names
    feature_names = list(features_raw.columns)
    feature_names.remove('global_id')
    feature_names.remove('timestep')
    feature_names.remove('section')
    feature_names.remove('spatial_id')
    feature_names.remove('x')
    feature_names.remove('y')
    # label_names = list(labels_raw.columns)
    log.write_line("additional values generated successfully.")
       
    # get integer classifier count
    classifier_count = getenv('NUM_CLASSIFIERS')
    classifier_count = int(classifier_count)
    log.write_line("got classifier count.")
    
    # create model class instance
    model = SSGModel(classifier_model, classifier_count, pred_start_timestep, log)
    log.write_line("model instantiated.")
    
    # train the model
    train_x, predict_x, store_columns = model.train(TRAIN_SECTION, int(getenv('TEST_SECTION')), features_raw, features, feature_names, labels, trap_effort, section_col, geo_features_csv)
    log.write_line("model trained successfully.")

    # log and make predictions for given input file range
    predictions, variances = model.predict(TRAIN_SECTION, train_x, predict_x, store_columns)
    log.write_line("predictions made successfully.")

    # add predictions to predictions directory
    predictions.to_csv(path.join(write_dir, 'predictions.csv'))
    log.write_line("predictions saved successfully.")

    # add variances of those predictions to the write directory if desired --> TODO Sus
    if variances:
        variances_csv_str = variances.to_csv(index=False, encoding='utf-8')
        variances_csv_str.to_csv(path.join(write_dir, 'variances.csv'))
        log.write_line("variances saved successfully.")

    # log completion
    log.write_line("predictions completed.")
    pass

In [20]:
## full API function ##

def invasive_species_trap_placement_api(log):
    data = traps_get_data(log)
    log.write_line("Finished grabbing data.")
    
    validated_data = traps_validate_data(log)
    log.write_line("Finished validating data.")

    preprocessed_data = traps_preprocess(log)
    log.write_line("Finished preprocessing data.")

    consolidated_data = traps_consolidate(log)
    log.write_line("Finished consolidating data.")

    predictions = traps_predict(log)
    log.write_line("Finished predicting trap locations.")

    return predictions

In [22]:
## run the code ##

# setup log
log = Log("../Logs/log.txt")
log.write_line("Initialized log.")

# run pipeline
predictions = invasive_species_trap_placement_api(log)
log.write_line("Finished predicting, closing log.")

## PAWS Implementations To Annotate

In [25]:
## IWARE MODEL FROM PAWS ##

import time
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.svm import LinearSVC, SVC
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.gaussian_process.kernels import RBF
# from iware.gpc import GaussianProcessClassifier

from os import getenv
from multiprocessing import Pool, get_context
from functools import partial
from os import getpid

# from opencensus.ext.azure import metrics_exporter
# from opencensus.stats import aggregation as aggregation_module
# from opencensus.stats import measure as measure_module
# from opencensus.stats import stats as stats_module
# from opencensus.stats import view as view_module
# from opencensus.tags import tag_map as tag_map_module
# from opencensus.tags import tag_key as tag_key_module
# from opencensus.tags import tag_value as tag_value_module

# appinsights_key = getenv('APPINSIGHTS_INSTRUMENTATIONKEY', None)

POSITIVE_LABEL = 1      # how a positive label is encoded in the data
RANDOM_SEED = None        # could be None
N_JOBS = 1 # -1 to use max

# parameters for bagging classifier
NUM_ESTIMATORS = 32 #32 #50
MAX_SAMPLES = 0.8
MAX_FEATURES = .5

# verbose output if == 1
VERBOSE = 0

MULTIPROCESSING_POOL_SIZE_ENV_VAR='MULTIPROCESSING_POOL_SIZE'

###########################################################
# utility functions
###########################################################
# given training and predict sets, normalize data to zero mean, unit variance
def normalize_data(train, predict):
    scaler = StandardScaler()
    # fit only on training data
    scaler.fit(train)
    # apply normalization to training and test data
    train = scaler.transform(train)
    predict = scaler.transform(predict)

    return train, predict

# by maximizing F1 score?
def determine_threshold(label, predict_test_pos_probs, num_thresholds=50):
    # TODO: previously, used tpr-(1-fpr)
    # fpr, tpr, thresholds = metrics.roc_curve(label, predict_test_pos_probs, pos_label=POSITIVE_LABEL)
    # or maybe scaled, like 2*tpr - (1-fpr)?

    thresholds = np.linspace(0, 1, num_thresholds)
    f1         = np.zeros(thresholds.size)
    precision  = np.zeros(thresholds.size)
    recall     = np.zeros(thresholds.size)
    auprc      = np.zeros(thresholds.size)

    for i in range(num_thresholds):
        predict_labels = predict_test_pos_probs > thresholds[i]
        predict_labels = predict_labels.astype(int)

        f1[i]        = metrics.f1_score(label, predict_labels)
        precision[i] = metrics.precision_score(label, predict_labels, pos_label=POSITIVE_LABEL)
        recall[i]    = metrics.recall_score(label, predict_labels, pos_label=POSITIVE_LABEL)

        precision_vals, recall_vals, _ = metrics.precision_recall_curve(label, predict_test_pos_probs, pos_label=POSITIVE_LABEL)
        auprc[i]     = metrics.auc(recall_vals, precision_vals)

        if VERBOSE:
            print('threshold: {:.4f} | f1: {:.4f},  precision: {:.4f}, recall: {:.4f}, AUPRC: {:.4f}'.format(thresholds[i], f1[i], precision[i], recall[i], auprc[i]))

    # opt = np.argmax(f1)
    opt = np.argmax(auprc)
    print('optimal threshold {:.4f}, with f1 {:.4f}, precision {:.4f}, recall {:.4f}, AUPRC {:.4f}'.format(thresholds[opt], f1[opt], precision[opt], recall[opt], auprc[opt]))

    return thresholds[opt]

# get classifier used as base estimator in bagging classifier
def get_base_estimator(method):
    if method == 'gaussian_processes':
        kernel = 1.0 * RBF(length_scale=1.0)
        base_estimator = GaussianProcessClassifier(kernel=kernel, random_state=RANDOM_SEED, warm_start=True, max_iter_predict=100, n_jobs=-1)
    elif method == 'svm':
        base_estimator = SVC(gamma='auto', random_state=RANDOM_SEED)
    elif method == 'linear-svc':
        base_estimator = LinearSVC(max_iter=5000, random_state=RANDOM_SEED)
    elif method == 'decision_tree':
        base_estimator = tree.DecisionTreeClassifier(random_state=RANDOM_SEED)
    else:
        raise Exception('method \'{}\' not recognized'.format(method))

    return base_estimator


# get overall classifier to use
def get_classifier(use_balanced, method):
    if method == 'random_forest':
        return RandomForestClassifier(n_estimators=NUM_ESTIMATORS,
            criterion='gini', max_depth=None, min_samples_split=2,
            min_samples_leaf=1, min_weight_fraction_leaf=0.0,
            max_features=MAX_FEATURES, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            bootstrap=True, oob_score=False, n_jobs=N_JOBS,
            random_state=RANDOM_SEED, verbose=VERBOSE,
            warm_start=False, class_weight=None)

    base_estimator = get_base_estimator(method)

    if method == 'gaussian_processes':
        # gaussian_processess don't need a bagging classifier
        return base_estimator
    elif use_balanced:
        # balanced bagging classifier used for datasets with strong label imbalance
        return BalancedBaggingClassifier(base_estimator=base_estimator,
            n_estimators=NUM_ESTIMATORS, max_samples=MAX_SAMPLES,
            max_features=MAX_FEATURES,
            bootstrap=True, bootstrap_features=False,
            oob_score=False, warm_start=False,
            sampling_strategy='majority', #sampling_strategy=0.8,
            replacement=True, n_jobs=N_JOBS,
            random_state=RANDOM_SEED, verbose=VERBOSE)
    else:
        # non-balanced bagging classifier used for other datasets
        return BaggingClassifier(base_estimator=base_estimator,
            n_estimators=NUM_ESTIMATORS, max_samples=MAX_SAMPLES,
            max_features=MAX_FEATURES,
            bootstrap=True, bootstrap_features=False,
            oob_score=False, warm_start=False, n_jobs=N_JOBS,
            random_state=RANDOM_SEED, verbose=VERBOSE)

def train_single_classifier(patrol_threshold, use_balanced, method, train_x, train_y, train_effort):
    # reduce to size desired
    train_y = np.squeeze(train_y)
    
    # idx = np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL))[0]

    # print('train_effort', train_effort)
    # print('patrol_threshold', patrol_threshold)
    # print('train_y', train_y)
    # print('train_y.shape', train_y.shape)
    # print('reshaped train_y', train_y)
    # print('reshaped train_y shape', train_y.shape)
    # print('POSITIVE_LABEL', POSITIVE_LABEL)

    # print('type', type(train_y))

    # print(np.where(train_effort >= patrol_threshold)[0].shape)
    # print(np.where(train_y == POSITIVE_LABEL)[0].shape)

    # print('np.logical or', np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL))
    # print('np.logical or shape', np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL).shape)
    # print('np.where', np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL)))
    # # print('np.where shape', np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL)).shape)
    # print('np.where[0] shape', np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL))[0].shape)
    # # print('np.where[1] shape', np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL))[1].shape)


    # idx_where_train_effort = np.where(train_effort >= patrol_threshold)[0]
    # print('idx_where_train_effort', idx_where_train_effort.shape)

    # idx_where_positive = np.where(train_y == POSITIVE_LABEL)[0]
    # print('idx_where_positive', idx_where_positive.shape)

    # get index of where to cut off points to analyze where either of these expressions holds (positive label or effort is beyond/at threshold)
    idx = np.where(np.logical_or(train_effort >= patrol_threshold, train_y == POSITIVE_LABEL))[0]
    
    # print('idx shape', idx.shape)
    # # print('patrol threshold', patrol_threshold)
    # print('train effort', train_effort.shape)
    # print('num positive train_y', np.sum(train_y))
    # print(' **** Training classifier for threshold: ' + str(patrol_threshold))
    # print("pid: ", getpid())

    # if idx has no points for a given training threshold nothing can happen
    if idx.size == 0:
        print('no training points found for threshold = {}'.format(patrol_threshold))
        return None

    # filter data for given idx 
    train_x_filter = train_x[idx, :]
    train_y_filter = train_y[idx]

    print('filtered data: {}. num positive labels {}. threshold {}'.format(train_x_filter.shape, np.sum(train_y_filter), patrol_threshold))

    # filtered labels have no positive hits, threshold not useful to train on.
    if np.sum(train_y_filter) == 0:
        print('no positive labels in this subset of the training data. skipping threshold {}'.format(patrol_threshold))
        return None

    # print('threshold {}, num x {}'.format(patrol_threshold, train_x_filter.shape))

    # # print('before raveled', train_y_filter.shape)
    # train_y_filter = train_y_filter.ravel() # to get rid of DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
    # # print('after raveled', train_y_filter.shape)
    # print('ravel train_y_filter done')

    # print('train x filter', train_x_filter)
    # print('train y filter', train_y_filter)

    # fit training data, get classifier and fit function according to method using get_classifier fn
    classifier = get_classifier(use_balanced, method)
    # print('get_classifier done')
    # print('train_x_filter shape', train_x_filter.shape)
    # print('train_y_filter shape', train_y_filter.shape)
    classifier.fit(train_x_filter, train_y_filter)

    # print('single classifier done fitting')

    # return fitted classifier
    return classifier


###########################################################
# iWare-E class
###########################################################
class iWare:
    def __init__(self, method, num_classifiers, year, task_id):
        self.method = method
        self.num_classifiers = num_classifiers
        self.year = year
        self.patrol_thresholds = None
        self.classifiers = None
        self.weights = None         # weights for classifiers

        self.classifier_train_measure = None
        self.stats_recorder = None
        self.key_patrol_threshold = None
        self.classifier_train_tmap = None

        # optional settings to monitor stats on runtime performance of classifier (training duration etc.)
        # if (appinsights_key):
        #     stats = stats_module.stats
        #     view_manager = stats.view_manager
        #     self.stats_recorder = stats.stats_recorder

        #     self.classifier_train_measure = measure_module.MeasureFloat("ClassifierTrainDuration",
        #                                                 "Classifier train duration in seconds",
        #                                                 "seconds")

        #     key_task_id = tag_key_module.TagKey("task_id")
        #     CLASSIFIER_TRAIN_VIEW = view_module.View("ClassifierTrainDuration",
        #                                         "Classifier train duration in seconds",
        #                                         [key_task_id],
        #                                         self.classifier_train_measure,
        #                                         aggregation_module.LastValueAggregation())

        #     self.classifier_train_tmap = tag_map_module.TagMap()
        #     self.classifier_train_tmap.insert(key_task_id, tag_value_module.TagValue(task_id))


    ###########################################################
    # classification
    ###########################################################

    def get_patrol_thresholds(self, train_effort):
        patrol_threshold_percentile = np.linspace(0, 100, self.num_classifiers, endpoint=False)
        patrol_thresholds = np.percentile(train_effort, patrol_threshold_percentile)
        print('percentiles {}'.format(patrol_threshold_percentile))
        print('patrol thresholds {}'.format(patrol_thresholds))
        return patrol_thresholds

    # currently does not use cross validation? or enable tuning of the V_p hyperparameter?
    # currently only does trivial case of identity matrix for the final combined matrix?
    def get_vote_matrix(self):
        vote_power = np.identity(self.num_classifiers)                           # identity matrix
        vote_qual = np.ones((self.num_classifiers, self.num_classifiers))

        # create combined vote matrix
        vote_combine = np.multiply(vote_power, vote_qual)

        # normalize column-wise
        vote_combine = vote_combine / vote_combine.sum(1)[:,None]

        return vote_combine




    # train a set of classifiers using provided data
    def train_classifiers(self, patrol_thresholds, train_x, train_y, train_effort, use_balanced):
        classifiers = []
        pool_size = getenv(MULTIPROCESSING_POOL_SIZE_ENV_VAR, None)
        print('pool size', pool_size)
        print('Training ' + str(len(patrol_thresholds)) + ' classifiers with a pool size of ' + str(pool_size))

        with get_context("spawn").Pool(processes=int(pool_size)) as pool:
            # time process
            start_time = time.time()
            # get classifiers by training each single classifier
            classifiers = pool.map(partial(train_single_classifier, use_balanced=use_balanced, method=self.method, train_x=train_x, train_y=train_y, train_effort=train_effort), patrol_thresholds)
            # get analytics: finish timing
            # if (appinsights_key):
            #     dur = (time.time() - start_time)
            #     print('Training duration: ' + str(dur))
            #     mmap = self.stats_recorder.new_measurement_map()
            #     mmap.measure_float_put(self.classifier_train_measure, dur)
            #     mmap.record(self.classifier_train_tmap)
        print('all train_single_classifiers done')
        # return results
        return classifiers

        # print('patrol thresholds', patrol_thresholds)
        # classifiers = []
        # for patrol_threshold in patrol_thresholds:
        #     trained_classifier = train_single_classifier(patrol_threshold, use_balanced, self.method, train_x, train_y, train_effort)
        #     print('trained classifier', trained_classifier)
        #     classifiers.append(trained_classifier)
        # print('all train_single_classifiers done')
        # return classifiers

    # training classifiers within the model
    def train_iware(self, all_train_x, all_train_y, all_train_effort, use_balanced=False, nsplits=5):
        self.patrol_thresholds = self.get_patrol_thresholds(all_train_effort)

        print('shape x', all_train_x.shape)
        print('shape y', all_train_y.shape)
        print('shape train_effort', all_train_effort.shape)

        self.weights = self.get_vote_matrix()

        print('-------------------------------------------')
        print('training classifiers with all train data')
        print('-------------------------------------------')

        self.classifiers = self.train_classifiers(self.patrol_thresholds, all_train_x, all_train_y, all_train_effort, use_balanced)
        print('done train_iware')

    ###########################################################
    # iWare-E for predicting future risk
    ###########################################################
    def train(self, predict_section, predict_section_test, features_raw, features, feature_names,
            labels, patrol_effort, section_col, input_static_feats,
            test_temp=None, test_precip=None, gaussian_processesp_filename=None):
        predict_year = self.year
        # ----------------------------------------------
        # get training data
        # ----------------------------------------------
        # use all data before specified (predict_year, predict_section)
        train_idx = np.where(np.logical_or(features_raw['year'] < predict_year,
            np.logical_and(features_raw['year'] == predict_year, features_raw['section'] < predict_section_test)))[0]

        train_x = features[train_idx, :]
        train_y = labels[train_idx]
        train_patrol_effort = patrol_effort[train_idx]

        # ----------------------------------------------
        # get data to predict on
        # ----------------------------------------------
        if predict_section == 0:
            prev_year = predict_year - 1
            num_section = np.max(section_col)
            prev_section = num_section
        else:
            prev_year = predict_year
            prev_section = predict_section - 1

        print('  test section: year {}, section {}'.format(predict_year, predict_section))
        print('  prev section: year {}, section {}'.format(prev_year, prev_section))

        # ----------------------------------------------
        # set up data arrays
        # ----------------------------------------------
        # get past patrol effort for the test section
        prev_section_idx = np.where(np.logical_and(features_raw['year'] == prev_year, features_raw['section'] == prev_section))
        past_patrol_effort = patrol_effort[prev_section_idx]

        prev_section_spatial_id = features_raw['spatial_id'].values[prev_section_idx]
        patrol_effort_df = pd.DataFrame({'spatial_id': prev_section_spatial_id,
                                            'past_patrol_effort': past_patrol_effort})

        # get all static features
        # input_static_featsa = list(input_static_feats.columns)
        # patrol_effort_dfa = list(patrol_effort_df.columns)


        #input_static_feats.drop(columns=input_static_feats.columns[0], inplace=True)
        # create features array and add in past_patrol_effort
        predict_x_df = input_static_feats.join(patrol_effort_df.set_index('spatial_id'), on='spatial_id', how='left')

        # input_static_featsa = list(predict_x_df.columns)

        predict_x_df['past_patrol_effort'].fillna(0, inplace=True)

        print(predict_x_df)

        # add climate info
        if test_temp is not None and test_precip is not None:
            predict_x_df['temp']   = test_temp * np.ones(input_static_feats.shape[0])
            predict_x_df['precip'] = test_precip * np.ones(input_static_feats.shape[0])

        # add gaussian_processes info
        if gaussian_processesp_filename is not None:
            new_gaussian_processesp = pd.read_csv('../preprocess_consolidate/belum_traponly_combined/1000/output/all_3month/gaussian_processesP_2019_0.csv')
            predict_x_df['gaussian_processesp'] = new_gaussian_processesp['2019-0']

        # arrange columns to match training data
        store_columns = predict_x_df[['spatial_id', 'x', 'y']]
        predict_x_df.drop(columns=['spatial_id', 'x', 'y'], inplace=True)

        predict_x_df = predict_x_df[feature_names]
        predict_x = predict_x_df.values

        # normalize data
        train_x, predict_x = normalize_data(train_x, predict_x)

        # ----------------------------------------------
        # train classifiers
        # ----------------------------------------------
        print('training classifiers on {} points...'.format(train_x.shape))

        train_start_time = time.time()
        self.train_iware(train_x, train_y, train_patrol_effort)
        total_train_time = time.time() - train_start_time
        print('total train time {:.3f}'.format(total_train_time))
        return train_x, predict_x, store_columns

    # use all provided data to make predictions
    def make_predictions(self, predict_section, train_x, predict_x, store_columns):
        # ----------------------------------------------
        # run classifiers to get set of predictions
        # ----------------------------------------------
        # intiialize array to store predictions from each classifier
        predict_year = self.year
        print('making predictions on year {} section {}... {} points'.format(predict_year, predict_section, predict_x.shape))
        final_predictions = np.zeros((predict_x.shape[0], self.num_classifiers))

        print('final_predictions', final_predictions)

        if self.method == 'gaussian_processes' or self.method == 'random_forest':
            final_variances = np.zeros((predict_x.shape[0], self.num_classifiers))

        print('self.num_classifiers', self.num_classifiers)

        print('0')

        # make predictions with each classifier
        for i in range(self.num_classifiers):

            print('')
            print('ITERATION', i)
            print('')

            # this classifier had no training points, so we skip it
            if self.classifiers[i] is None:
                final_predictions[:, i] = np.zeros((final_predictions.shape[0]))
                continue

            print('self.classifiers[i]', self.classifiers[i])

            if self.method == 'gaussian_processes' or self.method == 'random_forest':
                if self.method == 'random_forest':
                    curr_predictions, curr_variances = self.classifiers[i].predict_proba(predict_x, return_var=True, train_x=train_x)
                else:
                    curr_predictions = self.classifiers[i].predict_proba(predict_x)
                    curr_variances = self.classifiers[i].predict_var(predict_x)

                print('variance min {} max {}'.format(np.min(curr_variances), np.max(curr_variances)))

                final_variances[:, i] = curr_variances
            else:
                curr_predictions = self.classifiers[i].predict_proba(predict_x)


            # curr_predictions has two columns, first column for probability of 0, second col for prob of 1
            # if on classifier iter 8 and 9, number of filtered datapoints = number of positive illegal activity
            # this is because for higher patrol thresholds, there may not be any datapoints from train_x that have greater patrol effort
            # so for the np.where constructing filtered data, the selected data is just the illegal data,
            # so we can predict 100% confidence those have illegal instances
            # so for iter 8 and 9 it's just 1 column because the prob is all 1 so it'll be e.g. (97,1) rather than (97,2)

            # print('curr_predictions', curr_predictions)
            print(np.size(curr_predictions))

            # for (n,1) case, where all n probabilities are 1
            if np.size(curr_predictions) == curr_predictions.shape[0]:
                curr_predictions = np.ravel(curr_predictions)
            # for (n,2) case, first column for probability of 0, second col for prob of 1
            else:
                print('in')
                curr_predictions = curr_predictions[:, 1]   # probability of positive label

            final_predictions[:, i] = curr_predictions

            print('final_predictions', final_predictions)

        # save out predictions to CSV
        print('  save out predictions...')
        predictions_df = pd.DataFrame(data=final_predictions, columns=['threshold={}'.format(thresh) for thresh in self.patrol_thresholds])

        predictions_df = pd.concat([store_columns, predictions_df], axis=1)

        # save out variances to CSV
        variances_df = None
        if self.method == 'gaussian_processes' or self.method == 'random_forest':
            print('  save out variances...')
            variances_df = pd.DataFrame(data=final_variances, columns=['threshold={}'.format(thresh) for thresh in self.patrol_thresholds])
            variances_df = pd.concat([store_columns, variances_df], axis=1)
        return predictions_df, variances_df

ModuleNotFoundError: No module named 'pandas'

In [ ]:
## PREDICTION FROM PAWS ##
# predictions.py: defines functions to run predictions and do data setup within the prediction generation

# imports
# from iware.iware import * # custom package -> need to determine what to use here
# from iware import * # custom package -> need to determine what to use here
import calendar # default package
import copy # default package

# cuts useless stuff from big data frame before prediction
def setup_data(features_raw, labels_raw):
    # TODO mod to be trapping effort
    patrol_effort = features_raw['current_patrol_effort'].values
    section_col   = features_raw['section'].values
    year_col      = features_raw['year'].values

    # drop first column and the current_patrol_effort after grabbing the values and the corresponding first col label
    features_raw.drop(columns=features_raw.columns[0], inplace=True)
    features_raw.drop(columns='current_patrol_effort', inplace=True)
    labels_raw.drop(columns=labels_raw.columns[0], inplace=True)

    # get rid of following column labels 
    labels_raw.drop(columns='global_id', inplace=True)
    labels_raw.drop(columns='year', inplace=True)
    labels_raw.drop(columns='section', inplace=True)
    labels_raw.drop(columns='spatial_id', inplace=True)
    labels_raw.drop(columns='x', inplace=True)
    labels_raw.drop(columns='y', inplace=True)

    # make a copy of the features that removes the removed labels from ebfore
    features = copy.copy(features_raw)
    features.drop(columns='global_id', inplace=True)
    features.drop(columns='year', inplace=True)
    features.drop(columns='section', inplace=True)
    features.drop(columns='spatial_id', inplace=True)
    features.drop(columns='x', inplace=True)
    features.drop(columns='y', inplace=True)

    # grab the values from both edited objects 
    features = features.values
    labels   = labels_raw.values

    # convert column labels into a list to grab the feature names and remove the irrelevant feature names, grab label names
    feature_names = list(features_raw.columns)
    feature_names.remove('global_id')
    feature_names.remove('year')
    feature_names.remove('section')
    feature_names.remove('spatial_id')
    feature_names.remove('x')
    feature_names.remove('y')
    label_names = list(labels_raw.columns)

    # check feature and label names are converted properly
    print('feature names {}'.format(feature_names))
    print('label names {}'.format(label_names))

    # check shapes of all feature objects created are what we expected
    print('features_raw: ' + str(np.shape(features_raw)))
    print('features: ' + str(np.shape(features)))
    print('feature_names: ' + str(np.shape(feature_names)))
    # return necessary data
    return features_raw, features, feature_names, labels, patrol_effort, section_col, year_col



# making predictions runner function
# models accepted: decision_tree, random_forest, gaussian_processes
        # temporal_training_resolution_month_count: used to inform the number of datapoints in a month, tells us how many datapoints are attached to a given month
def run_predictions(TRAIN_SECTION, log, taskId,
                    dataIO_helper,
                    months_prefix,
                    temporal_training_resolution_month_count,
                    classifier_model,
                    prediction_start_year,
                    container_name,
                    run_id):
    # find number of sections
    section_count = int(12 / temporal_training_resolution_month_count)
    
    # gather the data into its final format
    get_top_directory = 'post_processed_data'
    path = '{}_{}month'.format(months_prefix, temporal_training_resolution_month_count)
    x_filename = '{}_x.csv'.format(months_prefix)
    features_raw = dataIO_helper.get_csv(x_filename, get_top_directory, path)

    path = '{}_{}month'.format(months_prefix, temporal_training_resolution_month_count)
    y_filename = '{}_y.csv'.format(months_prefix)
    labels_raw = dataIO_helper.get_csv(y_filename, get_top_directory, path)

    log.log_info('predict: {}'.format('Running setup_data.'), taskId=taskId)
    features_raw, features, feature_names, labels, patrol_effort, section_col, year_col = setup_data(features_raw, labels_raw)
    # get the classifier count 
    classifier_count = getenv('NUM_CLASSIFIERS')
    classifier_count = int(classifier_count)

    # create a model instance
    iware_model = iWare(classifier_model, classifier_count, prediction_start_year, taskId)

    # Train model #
    # get static features
    geo_features_csv = dataIO_helper.get_csv('static_features.csv', get_top_directory)

    # train the model
    train_x, predict_x, store_columns = iware_model.train(TRAIN_SECTION, int(getenv('TEST_SECTION')), features_raw, features, feature_names, labels, patrol_effort, section_col, geo_features_csv)

    # iterate over number of sections
    for section in range(1, section_count + 1):
        # find month range
        month_end_num = temporal_training_resolution_month_count * section
        month_end = calendar.month_name[month_end_num]
        month_start_num = month_end_num - (temporal_training_resolution_month_count - 1)
        month_start = calendar.month_name[month_start_num]

        # log and make predictions for given month range
        log.log_info('predict: {}'.format('Making predictions for section: {} - {}-{}'.format(str(section), month_start, month_end)), taskId=taskId)
        print('Making predictions for section: {} - {}-{}'.format(str(section), month_start, month_end))

        predictions, variances = iware_model.make_predictions(section, train_x, predict_x, store_columns)

        # add predictions to predictions directory
        write_dir = 'risk_prediction'
        if section_count == 1:
            dataIO_helper.write_csv(predictions, 'predictions_{}.csv'.format(prediction_start_year), write_dir)
        else:
            dataIO_helper.write_csv(predictions, 'predictions_{}-{}_{}.csv'.format(month_start, month_end, prediction_start_year), write_dir)

        # add variances of those predictions to the write directory if desired
        if variances:
            variances_csv_str = variances.to_csv(index=False, encoding='utf-8')
            if section_count == 1:
                dataIO_helper.write_csv(variances_csv_str, 'variances_{}.csv'.format(prediction_start_year), write_dir)
            else:
                month_end = calendar.month_name[temporal_training_resolution_month_count * section]
                month_start = calendar.month_name[month_end - temporal_training_resolution_month_count + 1]
                dataIO_helper.write_csv(variances_csv_str, 'variances_{}-{}_{}.csv'.format(month_start, month_end, prediction_start_year), write_dir)